# Loading Documents

----

In [ ]:
!pip install -qU langchain langchain-community langchain-openai langchain-text-splitters


In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("nvda_news_1.txt")

data = loader.load()

data[0].metadata

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader("movies.csv",source_column = 'title')

csv_data = loader.load()
len(csv_data)

In [ ]:
csv_data[3].metadata

In [ ]:
# Install unstructured for Colab (no magic-bin needed)
!pip install -q "unstructured[local-inference]" langchain-community

# Install system dependencies for PDF parsing
!apt-get update -qq
!apt-get install -qq poppler-utils tesseract-ocr

In [ ]:
from langchain_community.document_loaders import UnstructuredURLLoader

url_loader = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/banks/hdfc-bank-re-appoints-sanmoy-chakrabarti-as-chief-risk-officer-11259771.html",
        "https://www.moneycontrol.com/news/business/markets/market-corrects-post-rbi-ups-inflation-forecast-icrr-bet-on-these-top-10-rate-sensitive-stocks-ideas-11142611.html"

]
)

In [ ]:
url_data=url_loader.load()

len(url_data)

url_data[0].metadata